# Importing Data

In [139]:
import pandas as pd
import numpy as np


# read text sequence dataset
train_seq_df = pd.read_csv("./mini-project-1/datasets/train/train_text_seq.csv")
train_seq_X = train_seq_df['input_str'].tolist()
train_seq_Y = train_seq_df['label'].tolist()

valid_seq_df = pd.read_csv("./mini-project-1/datasets/valid/valid_text_seq.csv")
valid_seq_X = valid_seq_df['input_str'].tolist()
valid_seq_Y = valid_seq_df['label'].tolist()

test_seq_X = pd.read_csv("./mini-project-1/datasets/test/test_text_seq.csv")['input_str'].tolist()




print(f"Train dataset size: ")
print(f"train_seq_X: {len(train_seq_X)} train_seq_Y: {len(train_seq_Y)}")


print()
print("Test dataset size: ")
print(f"test_seq_X: {len(test_seq_X)} ")

Train dataset size: 
train_seq_X: 7080 train_seq_Y: 7080

Test dataset size: 
test_seq_X: 2232 


# Visual Observation Check

In [140]:
# Combining all training strings into one single string 

s = ''
for i in train_seq_X:
    s += i

In [141]:
# By visually inspecting the dataset, we see many 
# repeating blocks in these strings. 

a = dict()
l = []

for i in range(len(s)-5):
    if s[i:i+5] not in a:
        a[s[i:i+5]] = s.count(s[i:i+5])
        l.append((s.count(s[i:i+5]), s[i:i+5]))
        
l.sort(reverse=True)
l[:5]

[(7080, '15436'),
 (5388, '41596'),
 (4941, '15962'),
 (3707, '21596'),
 (3430, '61415')]

In [143]:
# '15436' has decent frequency and others lag behind quite a lot, so not considering them.

In [144]:
# Now we move on to see if any frequent 4 lenght substring occurs way too frequently.

a = dict()
l = []

for i in range(len(s)-4):
    if s[i:i+4] not in a:
        a[s[i:i+4]] = s.count(s[i:i+4])
        l.append((s.count(s[i:i+4]), s[i:i+4]))
        
l.sort(reverse=True)
l[:5]


[(14160, '1596'),
 (7134, '5436'),
 (7082, '1543'),
 (5580, '4262'),
 (5388, '4159')]

In [145]:
# '1596' has decent frequency and others lag behind quite a lot, so not considering them.
# Also, '5436' and '1543' are acctually subsrings of '15436' which is already considered above.

In [146]:
# Moving on to 3 digit substrings

a = dict()
l = []

for i in range(len(s)-3):
    if s[i:i+3] not in a:
        a[s[i:i+3]] = s.count(s[i:i+3])
        l.append((s.count(s[i:i+3]), s[i:i+3]))
        
l.sort(reverse=True)
l[:10]


[(14563, '262'),
 (14487, '614'),
 (14246, '596'),
 (14161, '159'),
 (7567, '415'),
 (7303, '436'),
 (7267, '154'),
 (7163, '422'),
 (7144, '543'),
 (7144, '284')]

In [147]:
# Among those not considered till now, 
# 262, 614, 

# Splitting input into subsequences

In [148]:
for i in range(len(train_seq_X)):
    train_seq_X[i] = str(int(train_seq_X[i]))

In [151]:
d = dict()

train_X_mod = []
train_Y_mod = []

for idx in range(len(train_seq_X)):
    st = str(int(train_seq_X[idx]))
    excess_dig = len(st) - 39
    ll = []
    

    while st:
        a5 = st[:5]
        a4 = st[:4]
        a3 = st[:3]

        c5 = s.count(a5)
        c4 = s.count(a4)
        c3 = s.count(a3)

        if len(ll) == 12:
            ll.append(st)
            st = ''

        elif len(st) <= 5:
            ll.append(st)
            st = ''

        elif 1.3*c5 > c4 and 1.2*c5 > c3 and excess_dig >= 2:
            ll.append(a5)
            st = st[5:]
            excess_dig -= 2
            to_add = a5
        elif 1.2*c4 > c3 and excess_dig >= 1:
            ll.append(a4)
            st = st[4:]
            excess_dig -= 1
            to_add = a4
        else:
            if len(st) == 3:
                ll.append(a3)
                st = st[3:]
                to_add = a3
            elif 1.2*(s.count(st[:3]) + s.count(st[3:6])) < s.count(st[:5]) + s.count(st[5:8]) and excess_dig >= 2:
                ll.append(a5)
                st = st[5:]
                to_add = a5
                excess_dig -= 2
            elif 1.2*(s.count(st[:3]) + s.count(st[3:6])) < s.count(st[:4]) + s.count(st[4:7]) and excess_dig >= 1:
                ll.append(a4)
                st = st[4:]
                to_add = a4
                excess_dig -= 1
            else:
                ll.append(a3)
                st = st[3:]
                to_add = a3

        if to_add in d:
            d[to_add] += 1
        else:
            d[to_add] = 1


    print(len(ll), ll)
    
    # print(len(ll), ll)
    
    if len(ll) == 13:
        train_X_mod.append(ll)
        train_Y_mod.append(train_seq_Y[idx])


13 ['15436', '464', '27181', '596', '614', '2800', '262', '422', '3132', '284', '1596', '262', '614']
13 ['464', '1596', '36951', '5436', '422', '262', '614', '11047', '1596', '262', '4761', '614', '284']
13 ['15436', '262', '1596', '5999', '614', '422', '464', '13580', '614', '262', '4051', '1596', '284']
13 ['15436', '422', '464', '12386', '14262', '1596', '895', '615', '962', '843', '510', '614', '262']
13 ['464', '1899', '422', '15436', '20690', '1596', '614', '262', '4761', '262', '1596', '614', '284']
13 ['1596', '614', '33658', '15436', '29194', '222', '624', '642', '8461', '415', '962', '780', '262']
13 ['614', '1596', '262', '422', '464', '8055', '5237', '15436', '614', '33651', '596', '284', '262']
13 ['5999', '422', '1596', '46485', '262', '621', '5436', '614', '1596', '262', '1495', '284', '614']
13 ['464', '15436', '30699', '1596', '422', '5705', '614', '262', '614', '1596', '262', '2414', '284']
13 ['262', '1596', '26390', '464', '422', '614', '15436', '2091', '4790', '26

In [152]:
len(train_X_mod), len(train_Y_mod)

(7080, 7080)

In [155]:
l1 = [str(int(i[0]))[:3] for i in train_X_mod]
l1 = list(set(l1))
l1.sort()
l1 = list(map(int, l1))
len(l1)

177

In [157]:
dv = dict()

test_X_mod = []
test_Y_mod = []

for idx in range(len(valid_seq_X)):
    st = str(int(valid_seq_X[idx]))
    excess_dig = len(st) - 39
    ll = []
    

    while st:
        a5 = st[:5]
        a4 = st[:4]
        a3 = st[:3]

        c5 = s.count(a5)
        c4 = s.count(a4)
        c3 = s.count(a3)

        if len(ll) == 12:
            ll.append(st)
            st = ''

        elif len(st) <= 5:
            ll.append(st)
            st = ''

        elif 1.1*c5 > c4 and 1.15*c5 > c3 and excess_dig >= 2:
            ll.append(a5)
            st = st[5:]
            excess_dig -= 2
            to_add = a5
        elif 1.2*c4 > c3 and excess_dig >= 1:
            ll.append(a4)
            st = st[4:]
            excess_dig -= 1
            to_add = a4
        else:
            if len(st) == 3:
                ll.append(a3)
                st = st[3:]
                to_add = a3
            elif 1.1*(s.count(st[:3]) + s.count(st[3:6])) < s.count(st[:5]) + s.count(st[5:8]) and excess_dig >= 2:
                ll.append(a5)
                st = st[5:]
                to_add = a5
                excess_dig -= 2
            elif 1.1*(s.count(st[:3]) + s.count(st[3:6])) < s.count(st[:4]) + s.count(st[4:7]) and excess_dig >= 1:
                ll.append(a4)
                st = st[4:]
                to_add = a4
                excess_dig -= 1
            else:
                ll.append(a3)
                st = st[3:]
                to_add = a3

        if to_add in dv:
            dv[to_add] += 1
        else:
            dv[to_add] = 1


    print(len(ll), ll)
    
    # print(len(ll), ll)
    
    if len(ll) == 13:
        test_X_mod.append(ll)
        test_Y_mod.append(valid_seq_Y[idx])


13 ['422', '15436', '262', '31596', '614', '34141', '262', '464', '284', '1596', '614', '262', '3901']
13 ['464', '614', '1250', '615', '436', '23272', '621', '596', '422', '1596', '262', '614', '2843104']
13 ['422', '4644', '5436', '15436', '262', '33656', '141', '596', '27132', '621', '596', '284', '614']
13 ['15436', '11201', '596', '43716', '142', '624', '224', '646', '143', '132', '262', '284', '1596']
13 ['33241', '596', '23684', '15436', '614', '464', '422', '262', '5705', '614', '284', '1596', '262']
13 ['422', '262', '464', '15436', '34141', '614', '19541', '596', '1596', '284', '1795', '614', '262']
13 ['614', '20752', '62464', '1596', '422', '21211', '543', '632', '706', '142', '622', '841', '596']
13 ['614', '1596', '15436', '422', '718', '43559', '262', '464', '284', '262', '614', '1596', '1731']
13 ['3901', '422', '1596', '10681', '15436', '614', '262', '464', '1596', '284', '2078', '262', '614']
13 ['614', '2598', '35121', '422', '15436', '262', '1596', '464', '1596', '2

In [158]:
len(test_X_mod)

489

In [159]:
# Use train_X_mod, train_Y_mod -> to train the model
# Use test_X_mod, test_Y_mod -> to validate